# PHW #3 - Chapters 4 & 5

Group: 1  
Author: Aleksandra Georgievska  
Date: October 4, 2023


### Chapter 04 - Subqueries - Exercises.sql


In [4]:
-- 6
-- Write a query that returns customers
-- who placed orders in 2015 but not in 2016
-- Tables involved: TSQLV4 database, Customers and Orders tables

-- PREDICATE: Use the EXISTS predicate to create 2 correlated queries that filters 
-- the customers who placed orders in 2015 but did not place orders in 2016. 

Use Northwinds2022TSQLV7

SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer] AS C
WHERE EXISTS 
    (SELECT *
    FROM Sales.[Order] AS O
    WHERE O.CustomerId = C.CustomerId
        AND O.OrderDate >= '20150101'
        AND O.OrderDate < '20160101')
    AND NOT EXISTS 
    (SELECT * 
    FROM Sales.[Order] as O
    WHERE O.CustomerId = C.CustomerId
        AND O.OrderDate >= '20160101'
        AND O.OrderDate < '20170101')


(7 rows affected)

Total execution time: 00:00:00.008

CustomerId,CustomerCompanyName
21,Customer KIDPX
23,Customer WVFAF
33,Customer FVXPQ
36,Customer LVJSO
43,Customer UISOJ
51,Customer PVDZC
85,Customer ENQZT


-- 9  
-- Explain the difference between IN and EXISTS

Answer:  

They are similar in that they both are used to filter and retrieve data based on specific conditions, but they operate differently.  

'IN' is used to filter rows based on a list of specific values by checking if a specified colunm's value matches any value in the list while 'EXISTS' is used to filter rows based on the existence of rows in a correlated subquery by checking if a subquery returns any rows, and if it does, it includes the rows from the outer query in the result. 


### <span style="font-size: 12pt; font-family: Arial, sans-serif; color: rgb(102, 102, 102); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; vertical-align: baseline; white-space: pre-wrap;">Chapter 04 - Subqueries.sql</span>

<span style="font-size: 12pt; font-family: Arial, sans-serif; color: rgb(102, 102, 102); background-color: transparent; font-variant-numeric: normal; font-variant-east-asian: normal; vertical-align: baseline; white-space: pre-wrap;"><br></span>

In [11]:
-- PREDICATE: Demonstrate an unwanted result of a null output when the 
-- intention was to show the two customers that didn't place orders 

---------------------------------------------------------------------
-- NULL Trouble
---------------------------------------------------------------------

-- Customers who didn't place orders

-- Using NOT IN
USE Northwinds2022TSQLV7

SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer]
WHERE CustomerId NOT IN(SELECT O.CustomerId
                    FROM Sales.[Order] AS O);


-- Add a row to the Orders table with a NULL custid
INSERT INTO Sales.[Order]
  (CustomerId, EmployeeId, OrderDate, RequiredDate, ShipToDate, ShipperId,
   Freight, ShipToName, ShipToAddress, ShipToCity, ShipToRegion,
   ShipToPostalCode, ShipToCountry)
  VALUES(NULL, 1, '20160212', '20160212',
         '20160212', 1, 123.00, N'abc', N'abc', N'abc',
         N'abc', N'abc', N'abc');

-- Following returns an empty set
SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer]
WHERE CustomerId NOT IN(SELECT O.CustomerId
                    FROM Sales.[Order] AS O);

(0 rows affected)

(1 row affected)

(0 rows affected)

Total execution time: 00:00:00.010

CustomerId,CustomerCompanyName


CustomerId,CustomerCompanyName


In [7]:
-- PREDICATE: Demonstrate the intended result of 2 outputs using the 
-- WHERE clause to show the two customers that didn't place orders 

-- Using NOT IN
USE Northwinds2022TSQLV7

SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer]
WHERE CustomerId NOT IN(SELECT O.CustomerId
                    FROM Sales.[Order] AS O);

-- Add a row to the Orders table with a NULL custid
INSERT INTO Sales.[Order]
  (CustomerId, EmployeeId, OrderDate, RequiredDate, ShipToDate, ShipperId,
   Freight, ShipToName, ShipToAddress, ShipToCity, ShipToRegion,
   ShipToPostalCode, ShipToCountry)
  VALUES(NULL, 1, '20160212', '20160212',
         '20160212', 1, 123.00, N'abc', N'abc', N'abc',
         N'abc', N'abc', N'abc');

-- Exclude NULLs explicitly (NEW)
SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer]
WHERE CustomerId NOT IN(SELECT O.CustomerId 
                    FROM Sales.[Order] AS O
                    WHERE O.CustomerId IS NOT NULL);

(0 rows affected)

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.016

CustomerId,CustomerCompanyName


CustomerId,CustomerCompanyName
22,Customer DTDMN
57,Customer WVAXS


In [12]:
-- PREDICATE: Demonstrate the intended result of 2 outputs using the 
-- NOT EXISTS predicate to show the two customers that didn't place orders 

-- Using NOT IN
USE Northwinds2022TSQLV7

SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer]
WHERE CustomerId NOT IN(SELECT O.CustomerId
                    FROM Sales.[Order] AS O);

-- Add a row to the Orders table with a NULL custid
INSERT INTO Sales.[Order]
  (CustomerId, EmployeeId, OrderDate, RequiredDate, ShipToDate, ShipperId,
   Freight, ShipToName, ShipToAddress, ShipToCity, ShipToRegion,
   ShipToPostalCode, ShipToCountry)
  VALUES(NULL, 1, '20160212', '20160212',
         '20160212', 1, 123.00, N'abc', N'abc', N'abc',
         N'abc', N'abc', N'abc');

-- Using NOT EXISTS
SELECT CustomerId, CustomerCompanyName
FROM Sales.[Customer] AS C
WHERE NOT EXISTS
  (SELECT * 
   FROM Sales.[Order] AS O
   WHERE O.CustomerId = C.CustomerId);

-- Cleanup
DELETE FROM Sales.[Order] WHERE CustomerId IS NULL;
GO


(0 rows affected)

(1 row affected)

(2 rows affected)

(6 rows affected)

Total execution time: 00:00:00.020

CustomerId,CustomerCompanyName


CustomerId,CustomerCompanyName
22,Customer DTDMN
57,Customer WVAXS


### <span style="background-color: transparent; color: rgb(102, 102, 102); font-family: Arial, sans-serif; font-size: 12pt; white-space: pre-wrap;">Chapter 05 - Table Expressions - Exercises.sql</span>

<span style="background-color: transparent; color: rgb(102, 102, 102); font-family: Arial, sans-serif; font-size: 12pt; white-space: pre-wrap;"><br></span>

In [23]:
-- 2-1
-- Write a query that returns the maximum order date for each employee
-- Tables involved: TSQLV4 database, Sales.Orders table

-- Desired output
-- empid       maxorderdate
-- ----------- -------------
-- 3           2016-04-30
-- 6           2016-04-23
-- 9           2016-04-29
-- 7           2016-05-06
-- 1           2016-05-06
-- 4           2016-05-06
-- 2           2016-05-05
-- 5           2016-04-22
-- 8           2016-05-06

-- (9 row(s) affected)

-- PREDICATE: Using the MAX built in function, show the most recent order date
-- for each employee in the Order table

USE Northwinds2022TSQLV7

SELECT EmployeeId, MAX(OrderDate) AS MaxOrderDate
FROM Sales.[Order]
GROUP BY EmployeeId
-- ORDER BY EmployeeId DESC


(9 rows affected)

Total execution time: 00:00:00.003

EmployeeId,MaxOrderDate
9,2016-04-29
3,2016-04-30
6,2016-04-23
7,2016-05-06
1,2016-05-06
4,2016-05-06
5,2016-04-22
2,2016-05-05
8,2016-05-06


In [17]:
-- 2-2
-- Encapsulate the query from exercise 2-1 in a derived table
-- Write a join query between the derived table and the Sales.Orders
-- table to return the Sales.Orders with the maximum order date for 
-- each employee
-- Tables involved: Sales.Orders

-- Desired output:
-- empid       orderdate   orderid     custid
-- ----------- ----------- ----------- -----------
-- 9           2016-04-29  11058       6
-- 8           2016-05-06  11075       68
-- 7           2016-05-06  11074       73
-- 6           2016-04-23  11045       10
-- 5           2016-04-22  11043       74
-- 4           2016-05-06  11076       9
-- 3           2016-04-30  11063       37
-- 2           2016-05-05  11073       58
-- 2           2016-05-05  11070       44
-- 1           2016-05-06  11077       65

-- (10 row(s) affected)

-- PREDICATE: Create a table to show the most recent order date for each 
-- employee and then use that derived table in an Inner Join between itself 
-- and the outer query using the Order table, matching the employee IDs
-- and the order date from the Order table with the Max order date from the
-- derived table 


USE Northwinds2022TSQLV7

SELECT O.EmployeeId, O.OrderDate, O.OrderId, O.CustomerId
FROM  Sales.[Order] AS O
    INNER JOIN (SELECT EmployeeId, MAX(OrderDate) AS MaxOrderDate
                FROM Sales.[Order]
                GROUP BY EmployeeId
                ) AS D
    ON O.EmployeeId = D.EmployeeId
    AND O.OrderDate = D.MaxOrderDate



-- When you’re done, run the following code for cleanup:
DROP VIEW IF EXISTS Sales.VEmpOrders;
DROP FUNCTION IF EXISTS Production.TopProducts;


(10 rows affected)

Total execution time: 00:00:00.004

EmployeeId,OrderDate,OrderId,CustomerId
9,2016-04-29,11058,6
8,2016-05-06,11075,68
7,2016-05-06,11074,73
6,2016-04-23,11045,10
5,2016-04-22,11043,74
4,2016-05-06,11076,9
3,2016-04-30,11063,37
2,2016-05-05,11073,58
2,2016-05-05,11070,44
1,2016-05-06,11077,65


### <span style="background-color: transparent; color: rgb(102, 102, 102); font-family: Arial, sans-serif; font-size: 12pt; white-space: pre-wrap;">Chapter 05 - Table Expressions.sql</span>

In [28]:
---------------------------------------------------------------------
-- Defining Multiple CTEs
---------------------------------------------------------------------

-- PREDICATE: Find the number of customers who placed orders in each year 
-- and then filter the results to show only the years with more than 70 customers
-- by building one CTE and then using those table results in CTE 2

USE Northwinds2022TSQLV7
GO

WITH C1 AS
(
  SELECT YEAR(OrderDate) AS OrderYear, CustomerId
  FROM Sales.[Order]
),
C2 AS
(
  SELECT OrderYear, COUNT(DISTINCT CustomerId) AS NumCusts
  FROM C1
  GROUP BY OrderYear
)
SELECT OrderYear, NumCusts
FROM C2
WHERE numcusts > 70;

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.003

OrderYear,NumCusts
2015,86
2016,81


In [31]:
---------------------------------------------------------------------
-- Multiple References
---------------------------------------------------------------------

-- PREDICATE: Using a CTE, get the annual customer counts in the Orders table
-- and calculate the growth in the number of customers who placed orders, 
-- year over year. 

USE Northwinds2022TSQLV7
GO

-- defining a CTE... 
WITH YearlyCount AS
(
  SELECT YEAR(OrderDate) AS OrderYear,
    COUNT(DISTINCT CustomerID) AS NumCusts
  FROM Sales.[Order]
  GROUP BY YEAR(OrderDate)
)
-- ... to use in the main query
SELECT Cur.OrderYear, 
  Cur.NumCusts AS CurNumCusts, Prv.NumCusts AS PrvNumCusts,
  Cur.NumCusts - Prv.NumCusts AS Growth
FROM YearlyCount AS Cur
  -- self join
  LEFT OUTER JOIN YearlyCount AS Prv
    ON Cur.OrderYear = Prv.OrderYear + 1;

Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.013

OrderYear,CurNumCusts,PrvNumCusts,Growth
2014,67,NULL,NULL
2015,86,67,19
2016,81,86,-5


In [32]:
---------------------------------------------------------------------
-- Recursive CTEs (Optional, Advanced)
---------------------------------------------------------------------

-- PREDICATE: Using UNION ALL, Recursively create the hierarchy of employee to 
-- manager relationships, designating Employee 2 as the base case in the recursion. 

Use Northwinds2022TSQLV7
GO 

-- CTE: 
WITH EmpsCTE AS
(
  -- base case or 'anchor member': 
  SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName
  FROM HumanResources.[Employee]
  WHERE EmployeeId = 2
  
  -- (combibe results of above and below queries):
  UNION ALL
  
  SELECT C.EmployeeId, C.EmployeeManagerId, C.EmployeeFirstName, C.EmployeeLastName
  -- EmpsCTE recursively calling itself: 
  FROM EmpsCTE AS P -- P for Parent
    INNER JOIN HumanResources.[Employee] AS C -- C for child
      ON C.EmployeeManagerId = P.EmployeeId
)
SELECT EmployeeId, EmployeeManagerId, EmployeeFirstName, EmployeeLastName
FROM EmpsCTE;

Commands completed successfully.

(8 rows affected)

Total execution time: 00:00:00.027

EmployeeId,EmployeeManagerId,EmployeeFirstName,EmployeeLastName
2,1,Don,Funk
3,2,Judy,Lew
5,2,Sven,Mortensen
6,5,Paul,Suurs
7,5,Russell,King
9,5,Patricia,Doyle
4,3,Yael,Peled
8,3,Maria,Cameron


In [33]:
---------------------------------------------------------------------
-- APPLY
---------------------------------------------------------------------

-- PREDICATE: Get the results of combining each row from the Shipper
-- table with every row from the Employee table using a cross join

USE Northwinds2022TSQLV7

SELECT S.ShipperId, E.EmployeeId
FROM Sales.[Shipper] AS S
  -- aka Cartesian join (cross product):
  CROSS JOIN HumanResources.[Employee] AS E;


-- PREDICATE: For each row in the  Shipper table, 'apply' the 
-- Employee table as if it were a table-valued function, resulting in 
-- all combinations of each shipper with the rows generated by applying the employee table

SELECT S.Shipperid, E.EmployeeId
FROM Sales.[Shipper] AS S
  -- CROSS APPLY applies a table-valued function or 
  -- subquery for each row in the first table
  CROSS APPLY HumanResources.[Employee] AS E;


-- PREDICATE: Retrieve the 3 most recent orders for each customer 
-- by using the CROSS APPLY predicate with a subquery on the Order table

-- 3 most recent orders for each customer
SELECT C.CustomerId, A.OrderId, A.OrderDate
FROM Sales.[Customer] AS C
  CROSS APPLY
    (SELECT TOP (3) OrderId, EmployeeId, OrderDate, RequiredDate 
     FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId
     ORDER BY OrderDate DESC, OrderId DESC) AS A;

(27 rows affected)

(27 rows affected)

(263 rows affected)

Total execution time: 00:00:00.010

ShipperId,EmployeeId
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,1


Shipperid,EmployeeId
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,1


CustomerId,OrderId,OrderDate
1,11011,2016-04-09
1,10952,2016-03-16
1,10835,2016-01-15
2,10926,2016-03-04
2,10759,2015-11-28
2,10625,2015-08-08
3,10856,2016-01-28
3,10682,2015-09-25
3,10677,2015-09-22
4,11016,2016-04-10


In [34]:
---------------------------------------------------------------------
-- APPLY (...Continued...)
---------------------------------------------------------------------

-- PREDICATE: Retrieve the 3 most recent orders for each customer by first
-- sorting the orders by date and ID in descending order, then using
-- OFFSET 0 ROWS FETCH NEXT 3 ROWS ONLY to obtain just the first three rows

Use Northwinds2022TSQLV7

-- With OFFSET-FETCH
SELECT C.CustomerId, A.OrderId, A.OrderDate
FROM Sales.[Customer] AS C
  CROSS APPLY
    (SELECT OrderId, EmployeeId, OrderDate, RequiredDate 
     FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId
     ORDER BY OrderDate DESC, OrderId DESC
     OFFSET 0 ROWS FETCH NEXT 3 ROWS ONLY) AS A;

(263 rows affected)

Total execution time: 00:00:00.008

CustomerId,OrderId,OrderDate
1,11011,2016-04-09
1,10952,2016-03-16
1,10835,2016-01-15
2,10926,2016-03-04
2,10759,2015-11-28
2,10625,2015-08-08
3,10856,2016-01-28
3,10682,2015-09-25
3,10677,2015-09-22
4,11016,2016-04-10


In [35]:
---------------------------------------------------------------------
-- APPLY (...Continued...)
---------------------------------------------------------------------

-- PREDICATE: Show all custumers, even those that didn't place orders by using
-- the OUTER clause, and for each customer, show the top 3 most recent orders

USE Northwinds2022TSQLV7
go

-- 3 most recent orders for each customer, preserve customers
SELECT C.CustomerId, A.OrderId, A.OrderDate
FROM Sales.[Customer] AS C
  OUTER APPLY
    (SELECT TOP (3) OrderId, EmployeeId, OrderDate, RequiredDate 
     FROM Sales.[Order] AS O
     WHERE O.CustomerId = C.CustomerId
     ORDER BY orderdate DESC, orderid DESC) AS A;

Commands completed successfully.

(265 rows affected)

Total execution time: 00:00:00.008

CustomerId,OrderId,OrderDate
1,11011,2016-04-09
1,10952,2016-03-16
1,10835,2016-01-15
2,10926,2016-03-04
2,10759,2015-11-28
2,10625,2015-08-08
3,10856,2016-01-28
3,10682,2015-09-25
3,10677,2015-09-22
4,11016,2016-04-10


In [36]:
---------------------------------------------------------------------
-- APPLY (...Continued...)
---------------------------------------------------------------------

-- PREDICATE: Define a function called TopOrders that selectes the top N orders (N is
-- an argument that gets passed in), and applies that function to each customer in the 
-- Customer table, effectively giving the top 3 orders for each customer as the result. 

USE Northwinds2022TSQLV7
GO

-- Creation Script for the Function TopOrders
DROP FUNCTION IF EXISTS dbo.TopOrders;
GO
CREATE FUNCTION dbo.TopOrders
  (@CustId AS INT, @n AS INT)
  RETURNS TABLE
AS
RETURN
  SELECT TOP (@n) OrderId, EmployeeId, OrderDate, RequiredDate 
  FROM Sales.[Order]
  WHERE CustomerId = @CustId
  ORDER BY OrderDate DESC, OrderId DESC;
GO

SELECT
  C.CustomerID, C.CustomerCompanyName,
  A.OrderId, A.EmployeeId, A.OrderDate, A.RequiredDate 
FROM Sales.[Customer] AS C
  CROSS APPLY dbo.TopOrders(C.CustomerId, 3) AS A;

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(263 rows affected)

Total execution time: 00:00:00.045

CustomerID,CustomerCompanyName,OrderId,EmployeeId,OrderDate,RequiredDate
1,Customer NRZBB,11011,3,2016-04-09,2016-05-07
1,Customer NRZBB,10952,1,2016-03-16,2016-04-27
1,Customer NRZBB,10835,1,2016-01-15,2016-02-12
2,Customer MLTDN,10926,4,2016-03-04,2016-04-01
2,Customer MLTDN,10759,3,2015-11-28,2015-12-26
2,Customer MLTDN,10625,3,2015-08-08,2015-09-05
3,Customer KBUDE,10856,3,2016-01-28,2016-02-25
3,Customer KBUDE,10682,3,2015-09-25,2015-10-23
3,Customer KBUDE,10677,1,2015-09-22,2015-10-20
4,Customer HFBZG,11016,9,2016-04-10,2016-05-08
